**Global Cancer Patients 데이터셋**

2015년부터 2024년까지 보고된 전 세계 암 환자 데이터를 포함하고 있으며, 암의 진단, 치료, 생존에 영향을 미치는 주요 요인들을 시뮬레이션한 데이터입니다.

제공된 학습용 데이터(cacner_train.csv)를 이용하여 암의 심각도(Severity)를 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(cancer_test.csv)에 적용하여 얻은 암의 심각도 예측 값을 아래 [제출형식]에 따라 csv 파일로 생성하여 제출하시오.
- 예측 결과는 F1 Score(macro) 평가지표에 따라 평가함
- 성능이 우수한 예측 모델을 구축하기 위해서는 데이터 정제, Feature Engineering, 하이퍼 파라미터(hyper parameter) 최적화, 모델 비교 등이 필요할 수 있음. 다만, 과적합에 유의하여야 함


[[제출 형식]]
- 가. CSV 파일명: result.csv(파일명에 디렉토리/폴더 지정불가
- 나. 예측 칼럼명 : pred
- 다. 제출 칼럼 개수 : pred 칼럼 1개
- 라. 평가용 데이터 개수와 예측 결과 데이터 개수 일치 : 2,193개

[[제공 데이터]]
- 데이터 목록
- cancer_train.csv : 학습용 데이터, 5,115개
- cancer_test.csv : 평가용 데이터, 2,193개
- 평가용 데이터는 'Severity' 칼럼 미제공

[[ CSV 파일 제출 방법 ]]
- CSV 파일명 : result.csv
- 암의 심각도 칼럼명 : pred
- 제출 CSV 파일 형식 예시
        pred
        High
        Low
        High
        Medium
        Medium

* 소수점 아래자릿수는 별도 지정하지 않음
* pred 칼럼 데이터 수는 2,193

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score

pd.set_option('display.max_column', 100)

# 성능평가 함수
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred1 = model.predict(x_train)
    y_pred2 = model.predict(x_test)
    C = f1_score(y_train, y_pred1, average="macro")
    D = f1_score(y_test, y_pred2, average="macro")
    return f"acc: {A:.4f}, {B:.4f}, F1: {C:.4f}, {D:.4f}"

path = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"
train = pd.read_csv(path + "cancer_train.csv")
test = pd.read_csv(path + "cancer_test.csv")
# print(train.head(5)) # 5115
# print(test.tail(5))  # 2193

# 데이터 전처리
X = train.drop(columns=['Severity'])
X_all = pd.concat([X, test])
columns_obj = X_all.select_dtypes(include='object').columns
for col in columns_obj:
    X_all[col] = LabelEncoder().fit_transform(X_all[col])
le = LabelEncoder()
Y = pd.Series(le.fit_transform(train['Severity']), name="Severity")
temp = StandardScaler().fit_transform(X_all)
X_all = pd.DataFrame(temp, columns=X_all.columns)

# 데이터 재분배
X = X_all.iloc[:len(X), :]
X_submission = X_all.iloc[len(X):, :]
# (5115, 14) (5115,) (2193, 14)
temp = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1234)
x_train, x_test, y_train, y_test = temp
# (4092, 14) (1023, 14) (4092,) (1023,)

# 모델생성
model1 = LogisticRegression().fit(x_train, y_train)
# print(get_scores(model1, x_train, x_test, y_train, y_test))
# acc: 0.9856, 0.9795, F1: 0.9858, 0.9795

# model2 = DecisionTreeClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test)) # acc: 1.0000, 0.8152, F1: 1.0000, 0.8214
# print(model2.get_depth()) # 14
# for d in range(3, 10):
#     model2 = DecisionTreeClassifier(max_depth=d, random_state=1234).fit(x_train, y_train)
#     print(f"{d}: ", get_scores(model2, x_train, x_test, y_train, y_test))
# 6:  acc: 0.8575, 0.8045, F1: 0.8615, 0.8122
model2 = DecisionTreeClassifier(max_depth=6, random_state=1234).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test))
# acc: 0.8575, 0.8045, F1: 0.8615, 0.8122

# model3 = RandomForestClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test)) # acc: 1.0000, 0.9267, F1: 1.0000, 0.9295
# for d in range(3, 10):
#     model3 = RandomForestClassifier(max_depth=d,random_state=1234).fit(x_train, y_train)
#     print(d, get_scores(model3, x_train, x_test, y_train, y_test))
# 4 acc: 0.8898, 0.8661, F1: 0.8942, 0.8705
model3 = RandomForestClassifier(max_depth=4,random_state=1234).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test))
# acc: 0.8898, 0.8661, F1: 0.8942, 0.8705

model4 = AdaBoostClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model4, x_train, x_test, y_train, y_test)) # acc: 0.7923, 0.7703, F1: 0.7871, 0.7653

model5 = GradientBoostingClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model5, x_train, x_test, y_train, y_test)) # acc: 0.8563, 0.8172, F1: 0.8605, 0.8227

# 모델선택, 제출파일 생성
f_model = model3
y_pred_num = f_model.predict(X_submission)
y_pred = le.inverse_transform(y_pred_num)
pd.DataFrame({'pred': y_pred}).to_csv("result_02(2기).csv", index=False)

# 제출파일 확인
temp = pd.read_csv("result_02(2기).csv")
print(temp.value_counts())
print(temp.value_counts(normalize=True))
Y = pd.Series(le.inverse_transform(Y), name="Severity")
print(Y[:len(X)].value_counts(normalize=True))

acc: 0.9905, 0.9433, F1: 0.9907, 0.9449
pred  
Medium    893
High      753
Low       547
Name: count, dtype: int64
pred  
Medium    0.407205
High      0.343365
Low       0.249430
Name: proportion, dtype: float64
Severity
Medium    0.410948
High      0.335875
Low       0.253177
Name: proportion, dtype: float64
